## 데이터 셋 로딩 테스트

In [ ]:
df = pd.read_csv(s3_amz_review_apparel_negative_path, nrows= 14000, 
                 encoding='utf-8',engine='python')
# df = pd.read_csv(s3_amz_review_apparel_negative_path,error_bad_lines=False,
#                  encoding='utf-8',engine='python')

# df = pd.read_csv(s3_amz_review_apparel_negative_path,error_bad_lines=False,engine='python')




# df = pd.read_csv(s3_amz_review_apparel_negative_path, nrows= 8936, quoting=csv.QUOTE_NONE,error_bad_lines=False,
#                  encoding='utf-8',engine='python')
# df = pd.read_csv(s3_amz_review_apparel_negative_path, nrows= 8936, error_bad_lines=False,
#                  encoding='utf-8',engine='python')



## Download AMAZON Review 

In [21]:
import csv

df = pd.read_csv(output_infer_path, 
                 delimiter='\t', 
                 quoting=csv.QUOTE_NONE,
                 compression='gzip')
df.shape

(5906333, 15)

In [34]:
import os, sagemaker

def download_amazon_review(basic_folder = 's3://amazon-reviews-pds/tsv', prefix='', down_folder=''):
    s3_source_path = os.path.join(basic_folder, prefix)
#    print(s3_source_path)
    sagemaker.s3.S3Downloader.download(s3_source_path, down_folder)
    print(f'{s3_source_path} is downloaded')


amazon_data = 'az_data'
os.makedirs(amazon_data, exist_ok=True)

prefix = 'amazon_reviews_us_Apparel_v1_00.tsv.gz'
download_amazon_review(prefix = prefix, down_folder = amazon_data)
prefix = 'amazon_reviews_us_Beauty_v1_00.tsv.gz'
download_amazon_review(prefix = prefix, down_folder = amazon_data)
prefix = 'amazon_reviews_us_Grocery_v1_00.tsv.gz'
download_amazon_review(prefix = prefix, down_folder = amazon_data)

s3://amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz is downloaded
s3://amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz is downloaded
s3://amazon-reviews-pds/tsv/amazon_reviews_us_Grocery_v1_00.tsv.gz is downloaded


In [35]:
# ! aws s3 ls 's3://amazon-reviews-pds/tsv' --recursive

## CounterVectorizer

In [9]:
# !pip install konlpy

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

import numpy as np
np.random.seed(0)
 
from konlpy.tag import Twitter
twitter = Twitter()
 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
 
# tokenizer : 문장에서 색인어 추출을 위해 명사,동사,알파벳,숫자 정도의 단어만 뽑아서 normalization, stemming 처리하도록 함
def tokenizer(raw, pos=["Noun","Alpha","Verb","Number"], stopword=[]):
    return [
        word for word, tag in twitter.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅋ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]
# 테스트 문장
rawdata = [
    '남북 고위급회담 대표단 확정..남북 해빙모드 급물살',
    '[남북 고위급 회담]장차관만 6명..판 커지는 올림픽 회담',
    
    '문재인 대통령과 대통령의 영부인 김정숙여사 내외의 동반 1987 관람 후 인터뷰',
    '1987 본 문 대통령.."그런다고 바뀌나? 함께 하면 바뀐다"',
    
    '이명박 전 대통령과 전 대통령의 부인 김윤옥 여사, 그리고 전 대통령의 아들 이시형씨의 동반 검찰출석이 기대됨'
]

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [81]:
from IPython.display import display

In [8]:
vectorize = CountVectorizer(
    tokenizer=tokenizer, 
    min_df=2    # 예제로 보기 좋게 1번 정도만 노출되는 단어들은 무시하기로 했다
                # min_df = 0.01 : 문서의 1% 미만으로 나타나는 단어 무시
                # min_df = 10 : 문서에 10개 미만으로 나타나는 단어 무시
                # max_df = 0.80 : 문서의 80% 이상에 나타나는 단어 무시
                # max_df = 10 : 10개 이상의 문서에 나타나는 단어 무시
)
 
# 문장에서 노출되는 feature(특징이 될만한 단어) 수를 합한 Document Term Matrix(이하 DTM) 을 리턴한다
X = vectorize.fit_transform(rawdata)
 
print(
    'fit_transform, (sentence {}, feature {})'.format(X.shape[0], X.shape[1])
)
# fit_transform, (sentence 5, feature 7)
 
print(type(X))
# <class 'scipy.sparse.csr.csr_matrix'>
 
print(X.toarray())

fit_transform, (sentence 5, feature 7)
<class 'scipy.sparse.csr.csr_matrix'>
[[0 1 2 0 0 0 1]
 [0 1 1 0 0 0 2]
 [1 0 0 2 1 1 0]
 [1 0 0 1 0 0 0]
 [0 0 0 3 1 1 0]]


In [10]:
features = vectorize.get_feature_names()

In [11]:
features

['1987', '고위', '남북', '대통령', '동반', '여사', '회담']

In [83]:
df = pd.DataFrame(data=X.toarray(), columns=features)
df

,1987,고위,남북,대통령,동반,여사,회담
0,0,1,2,0,0,0,1
1,0,1,1,0,0,0,2
2,1,0,0,2,1,1,0
3,1,0,0,1,0,0,0
4,0,0,0,3,1,1,0


## 네이버 영화 리뷰

In [ ]:
# !wget -O train.txt https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
# !wget -O test.txt https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

train_df = pd.read_csv('train.txt', sep='\t')
print("train_df shape: ", train_df.shape)
test_df = pd.read_csv('test.txt', sep='\t')
print("test_df shape: ", test_df.shape)
